# Training a model on CarbonTracker's carbon flux
This notebook outlines the entire workflow to load and preprocess the following data sets, to be able to train a ML model:

- CarbonTracker
- ERA5 (monthly)
- ERA5-land (monthly)
- SPEI (monthly)
- MODIS (monthly)
- Biomass (yearly)
- Copernicus Landcover (yearly)

First follow the data downloading and config setup instuctions.

If you run this notebook on Surf Research Cloud, you shouldn't need to do this anymore.

In [1]:
from pathlib import Path

from dask.distributed import Client

from excited_workflow import carbon_tracker


client = Client()

Define the paths for the carbon tracker dataset, the regions dataset and output directory, as well as define the datasets that need to be included, the input variables (x_keys) for the model and the target variable (y_key).

In [2]:
cb_file= Path("/data/volume_2/EXCITED_prepped_data/CT2022.flux1x1-monthly.nc")
regions_file = Path("/data/volume_2/EXCITED_prepped_data/regions.nc")
output_path = Path("/home/cdonnelly")

desired_data = [
    "biomass",
    "spei",
    "modis",
    "era5_monthly",
    "era5_land_monthly",
    "copernicus_landcover"
]

x_keys = ["d2m", "mslhf", "msshf", "ssr", "str", "t2m", "spei", "NIRv", "skt",
            "stl1", "swvl1", "lccs_class"]
y_key = "bio_flux_opt"

Merge the desired datasets into a single xr.Dataset with the same dimensions as the carbon tracker dataset. 

In [3]:
ds_input = carbon_tracker.merge_datasets(desired_data, cb_file)

To limit the analyis to Transcom region 2 (North America) we require the `regions.nc` file:

In [4]:
ds_na = carbon_tracker.mask_region(regions_file, cb_file, "transcom_regions", ds_input)
ds_na


<xarray.Dataset>
Dimensions:           (time: 240, latitude: 180, longitude: 360)
Coordinates:
  * longitude         (longitude) float64 -179.5 -178.5 -177.5 ... 178.5 179.5
  * latitude          (latitude) float64 -89.5 -88.5 -87.5 ... 87.5 88.5 89.5
  * time              (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2019-12-01
Data variables: (12/26)
    bio_flux_opt      (time, latitude, longitude) float64 nan nan ... nan nan
    transcom_regions  (latitude, longitude) float64 nan nan nan ... nan nan nan
    biomass           (time, latitude, longitude) float64 nan nan ... nan nan
    spei              (time, latitude, longitude) float64 nan nan ... nan nan
    NDVI              (time, latitude, longitude) float64 nan nan ... nan nan
    NIRv              (time, latitude, longitude) float64 nan nan ... nan nan
    ...                ...
    stl4              (time, latitude, longitude) float64 nan nan ... nan nan
    swvl1             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl2             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl3             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl4             (time, latitude, longitude) float64 nan nan ... nan nan
    lccs_class        (time, latitude, longitude) float32 nan nan ... nan nan
Attributes:
    averaging_period_length_hours:  744
    email:                          carbontracker.team@noaa.gov
    url:                            http://carbontracker.noaa.gov
    institution:                    NOAA Earth System Research Laboratory
    Conventions:                    CF-1.5
    history:                        Time-stamp: <Orion-login-1.HPC.MsState.Ed...
    NCO:                            netCDF Operators version 4.9.3 (Homepage ...
    version:                        CT2022 1x1 3-hourly fluxes as of 2023-01-...

Validate the model by splitting the dataset into 5 groups. Train the model over 4 groups and predict over the remaining group iteratively. Output rmse netcdfs and scatterplots are stored in the output directory.

In [5]:
carbon_tracker.validate_model(ds_na, 5, x_keys, y_key, output_path)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.0000003,0.0000000,0.0000005,0.7766024,0.0000005,4.1166476,0.7830000


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.0000003,0.0000000,0.0000005,0.7634546,0.0000005,5.6257488,0.4100000


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.0000003,0.0000000,0.0000005,0.7671847,0.0000005,4.3469085,0.4080000


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.0000003,0.0000000,0.0000005,0.7627869,0.0000005,5.3898028,0.4130000


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.0000003,0.0000000,0.0000005,0.7572602,0.0000005,4.7043318,0.4080000


Train the model over the entire dataset and save the model to ONNX in the output directory. 

In [6]:
onnx_model = carbon_tracker.save_model(ds_na, x_keys, y_key, output_path)

2023-12-11 16:42:50,590 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.0000003,0.0000000,0.0000005,0.7653452,0.0000005,4.6548117,0.4980000


2023-12-11 16:42:56,649 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
2023-12-11 16:42:58,485 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
The maximum opset needed by this model is only 8.


Create dataframe to run the model with. 

In [7]:
df = ds_na.to_dataframe().dropna()
df = df[x_keys]
df.head()

d2m       mslhf      msshf         ssr  \
time       latitude longitude                                                  
2000-08-01 16.5     -97.5      291.160736 -123.552444 -29.256828  17632392.0   
                    -96.5      287.451385  -98.636139 -44.350342  17875046.0   
                    -95.5      292.602661  -88.933502 -76.770813  18379498.0   
           17.5     -100.5     290.568329 -118.364532 -51.205177  17707054.0   
                    -99.5      290.078156 -119.800644 -42.374344  17862124.0   

                                     str         t2m      spei      NIRv  \
time       latitude longitude                                              
2000-08-01 16.5     -97.5     -4539167.0  294.455078  0.402637  0.264352   
                    -96.5     -5688396.0  291.743347  0.859432  0.186314   
                    -95.5     -4277693.5  298.020874 -0.002091  0.245570   
           17.5     -100.5    -3109625.0  293.402252  0.364373  0.270905   
                    -99.5     -3914028.0  293.219940  0.007593  0.249498   

                                      skt        stl1     swvl1  lccs_class  
time       latitude longitude                                                
2000-08-01 16.5     -97.5      293.800140  294.488708  0.405656        90.0  
                    -96.5      291.346344  292.329987  0.441099       120.0  
                    -95.5      298.976379  299.503906  0.340303        60.0  
           17.5     -100.5     293.096253  293.790924  0.412808        90.0  
                    -99.5      291.973236  292.737579  0.406332       120.0

Open model and run it over the dataframe to check it was saved correctly. 

In [8]:
from onnxruntime import InferenceSession
import datetime

time = datetime.datetime.now().strftime('%Y-%m-%d_%H')
output_dir = output_path / f"carbon_tracker-{time}/lightgbm.onnx"
with open(output_dir, "rb") as f:
    model = f.read()

sess = InferenceSession(model)
predictions_onnx = sess.run(None, {'X': df.to_numpy()})[0]

In [9]:
predictions_onnx

array([[-1.2601057e-06],
       [-6.7932012e-07],
       [-4.9193414e-07],
       ...,
       [ 4.4452591e-07],
       [ 4.4147706e-07],
       [ 1.4241760e-07]], dtype=float32)